In [1]:
import pandas as pd
import os

In [2]:
classes_data = pd.read_csv('/home/ryzenrtx/CBCT_YOLO/men_women/class-descriptions-boxable.csv')
classes_data.columns

Index(['id', 'classes'], dtype='object')

In [30]:
class_list = dict()
for id, clas in zip(classes_data['id'],classes_data['classes']):
    if clas =='Woman' or clas == "Man": # Get the id of the class you want to annotate and add it as or condition
        class_list[clas] = id
class_list

{'Woman': '/m/03bt1vf', 'Man': '/m/04yx4'}

In [32]:
classes = list(class_list.keys())
classes

['Woman', 'Man']

In [35]:
class_string = [class_list[key] for key in class_list]
class_string

['/m/03bt1vf', '/m/04yx4']

In [30]:
annotation_data = pd.read_csv('/home/ryzenrtx/CBCT_YOLO/men_women/train-annotations-bbox.csv', usecols=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']) # Path of annotation file
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [31]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_string)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
24,000002c707c9895e,/m/03bt1vf,0.053085,0.464849,0.319846,0.839114
50,000004f4400f6ec5,/m/04yx4,0.138125,0.621250,0.217554,0.999066
51,000004f4400f6ec5,/m/04yx4,0.404375,0.788750,0.171802,0.999066
52,000004f4400f6ec5,/m/04yx4,0.615000,0.999375,0.656396,0.999066
84,0000201cd362f303,/m/03bt1vf,0.250000,0.395000,0.236398,0.729831


In [32]:
filtered_class_data['ClassNumber'] = ''
filtered_class_data['center x'] = ''
filtered_class_data['center y'] = ''
filtered_class_data['width'] = ''
filtered_class_data['height'] = ''

In [34]:
for i in range(len(class_string)):
    filtered_class_data.loc[filtered_class_data['LabelName'] == class_string[i], 'ClassNumber'] = i

filtered_class_data['center x'] = (filtered_class_data['XMax'] + filtered_class_data['XMin'])/2
filtered_class_data['center y'] = (filtered_class_data['YMax'] + filtered_class_data['YMin'])/2

filtered_class_data['width'] = filtered_class_data['XMax'] - filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax'] - filtered_class_data['YMin']

YOLO_values = filtered_class_data.loc[:, ['ImageID', 'ClassNumber', 'center x', 'center y', 'width', 'height']].copy()
YOLO_values.shape

(2185931, 6)

In [39]:
YOLO_values

,ImageID,ClassNumber,center x,center y,width,height
24,000002c707c9895e,1,0.258967,0.579480,0.411764,0.519268
50,000004f4400f6ec5,0,0.379687,0.608310,0.483125,0.781512
51,000004f4400f6ec5,0,0.596562,0.585434,0.384375,0.827264
52,000004f4400f6ec5,0,0.807187,0.827731,0.384375,0.342670
84,0000201cd362f303,1,0.322500,0.483115,0.145000,0.493433
...,...,...,...,...,...,...
14610164,ffffd9716fd38279,0,0.384062,0.686667,0.256875,0.625000
14610169,ffffda81903d6bb7,0,0.056875,0.787084,0.113750,0.424167
14610170,ffffda81903d6bb7,0,0.296250,0.599167,0.271250,0.800000
14610171,ffffda81903d6bb7,0,0.905625,0.390834,0.187500,0.203333


In [ ]:
img_path = "/home/ryzenrtx/CBCT_YOLO/OIDv4_ToolKit/OID/Dataset/validation/Woman" # Path of images
labels_path = "/home/ryzenrtx/CBCT_YOLO/OIDv4_ToolKit/OID/Dataset/validation/labels_women" # Path of labels folder
os.chdir(img_path)

for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.jpg'):
            img_title = f[:-4]
            print(img_title)
            YOLO_files = YOLO_values.loc[YOLO_values['ImageID'] == img_title]
            df = YOLO_files.loc[:, ['ClassNumber', 'center x', 'center y', 'width', 'height']].copy()

            save_path = labels_path + '/' + img_title + '.txt'

            df.to_csv(save_path, sep=' ', index=False, header=False)